<a href="https://colab.research.google.com/github/u2200579/Bot-Detection/blob/main/BotDetection2017Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 13.8 MB/s 


In [3]:
pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 15.6 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=82cc8bc1f633f777c87ea462905af7f51dc3ff9953a952bd0de57b79c2303b4f
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [4]:
import numpy as np
import pandas as pd
import emoji
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from sklearn.metrics.pairwise import cosine_similarity
import gensim
import gensim.downloader
import nltk  
nltk.download('punkt')
from tqdm import tqdm
tqdm.pandas()
from nltk.tokenize import word_tokenize
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
!cp "/content/drive/My Drive/genuineusers.csv" "genuineusers.csv" #Import user Profile information for geneuine users and bots
!cp "/content/drive/My Drive/SS1users.csv" "SS1users.csv"
!cp "/content/drive/My Drive/SS2users.csv" "SS2users.csv"
!cp "/content/drive/My Drive/SS3users.csv" "SS3users.csv"
!cp "/content/drive/My Drive/TS1users.csv" "TS1users.csv"

In [6]:
!cp "/content/drive/My Drive/Dataset1Tweets.csv" "tweets.csv" #Import Tweet information for geneuine users and bots

In [56]:
dfs = []

dfs.append(pd.read_csv("SS1users.csv",parse_dates=['created_at']))
dfs.append(pd.read_csv("SS2users.csv",parse_dates=['created_at']))
dfs.append(pd.read_csv("SS3users.csv",parse_dates=['created_at']))
dfs1 = pd.read_csv("TS1users.csv",parse_dates=['created_at'])
dfs1['created_at'] = dfs1['created_at'].str.replace(r'\D+', '', regex=True).astype('int')
dfs1['created_at'] = pd.to_datetime(dfs1['created_at'],unit='ms')
dfs1['is_bot'] = 1
dfs1['Data'] = 'TraditionalSocialSpambot'

botsCsv = pd.concat(dfs, axis=0, sort=True)
botsCsv['is_bot'] = 1
botsCsv['Data'] = 'SocialSpambot'
print("Fake users done!")
dfs2 = []
dfs2.append(pd.read_csv("genuineusers.csv",parse_dates=['created_at']))

print("Human users done!")

humansCsv = pd.concat(dfs2, sort=True)
humansCsv['is_bot']=0
humansCsv['Data']='Human'

resultUsers = pd.concat([botsCsv, dfs1, humansCsv], sort=True)
resultUsers['created_at'] = pd.to_datetime(resultUsers['created_at'],utc=True)
resultUsers['crawled_at'] = pd.to_datetime(resultUsers['crawled_at'],utc=True)
users = pd.DataFrame()
users['id']=resultUsers['id']
users['status_count'] = resultUsers['statuses_count']
users['friends_count'] = resultUsers['friends_count']
users['favorites_count'] = resultUsers['favourites_count']
users['followers_count'] = resultUsers['followers_count']
users['friends_to_followers_ratio'] = resultUsers['friends_count']/resultUsers['followers_count']
users['listed_count'] = resultUsers['listed_count']
users['is_bot'] = resultUsers['is_bot']
users['Account_age'] = (resultUsers['crawled_at'] - resultUsers['created_at']).dt.days
users['Data'] = resultUsers['Data']
users['description'] = resultUsers['description']
users['screen_name'] = resultUsers['screen_name']
users['name'] = resultUsers['name']

Fake users done!
Human users done!


In [8]:
df = pd.read_csv("tweets.csv") #datasets of bots and genuine accounts

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
df = df.head()
df

,contributors,crawled_at,created_at,favorite_count,favorited,geo,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,reply_count,retweet_count,retweeted,retweeted_status_id,source,text,timestamp,truncated,updated,user_id
0,NaN,2014-11-12 21:44:09,Wed Nov 12 20:14:48 +0000 2014,0.0,NaN,NaN,532627591686275072,NaN,0.0,0.0,...,0.0,0.0,NaN,0.0,"<a href=""http://www.facebook.com/twitter"" rel=...",I Pooh - In silenzio 1968 http://t.co/ahvQxUqTws,2014-11-12 21:14:48,NaN,2014-11-12 21:44:09,24858289.0
1,NaN,2014-11-12 21:44:09,Wed Nov 12 20:01:32 +0000 2014,0.0,NaN,NaN,532624255058706432,NaN,0.0,0.0,...,0.0,0.0,NaN,0.0,"<a href=""http://www.facebook.com/twitter"" rel=...",http://t.co/HyI5EQKz6Q,2014-11-12 21:01:32,NaN,2014-11-12 21:44:09,24858289.0
2,NaN,2014-11-12 21:44:09,Wed Nov 12 12:41:32 +0000 2014,0.0,NaN,NaN,532513524460052480,NaN,0.0,0.0,...,0.0,0.0,NaN,0.0,"<a href=""http://www.facebook.com/twitter"" rel=...","Tutti a tavola, con il filetto di baccalÃ . ht...",2014-11-12 13:41:32,NaN,2014-11-12 21:44:09,24858289.0
3,NaN,2014-11-12 21:44:09,Tue Nov 11 22:23:43 +0000 2014,0.0,NaN,NaN,532297646669852672,NaN,0.0,0.0,...,0.0,0.0,NaN,0.0,"<a href=""http://www.facebook.com/twitter"" rel=...",http://t.co/NAHQ4l2pUy,2014-11-11 23:23:43,NaN,2014-11-12 21:44:09,24858289.0
4,NaN,2014-11-12 21:44:09,Tue Nov 11 22:17:01 +0000 2014,0.0,NaN,NaN,532295960807100416,NaN,0.0,0.0,...,0.0,0.0,NaN,0.0,"<a href=""http://www.facebook.com/twitter"" rel=...",Gold - Spandau Ballet http://t.co/o8ZJHt7Neu,2014-11-11 23:17:01,NaN,2014-11-12 21:44:09,24858289.0


In [49]:
df1 = df[['user_id','num_hashtags','num_urls','num_mentions','retweet_count','is_bot']] #create dataframe containing tweet information
df2 = df[['user_id','text']] #create dataframe for tweets to calcukate semantic analysis

In [11]:
sid_obj = SentimentIntensityAnalyzer() #variable for sentiment calculator
def subject_calc(text): #function to calculate subjectivity of tweets
    try:
        return TextBlob(text).sentiment.subjectivity 
    except:
        return 0

def sentiment_calc(text): #function to calculate sentiment of tweets
    try:
        return  sid_obj.polarity_scores(text)
    except:
        return 0
        
def mean_embeddings(s):
    """Transfer a list of words into mean embedding"""
    return np.mean([glove_vec.get_vector(x) for x in s if x in glove_vec], axis=0) #function to extract word embedding for words in tweetsd embedding for words in tweets

In [ ]:
df2['subjectivity_score'] = df2['text'].apply(subject_calc) #Apply function calculating subjectivity of an tweet to each tweet 
df2['sentiment_score'] = df2['text'].apply(sentiment_calc) #Apply function calculating sentiment scores of an tweet to each tweet
df2 = pd.concat([df2.drop(['sentiment_score'], axis=1), df2['sentiment_score'].apply(pd.Series)], axis=1) 

In [13]:
glove_vec = gensim.downloader.load('glove-twitter-25') #upload glove embeddings

[==================================================] 100.0% 104.8/104.8MB downloaded


In [14]:
import operator  #used to build vocabulary to check similarity between word embeddings and tweet tokens


def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab
df2['text']=df2['text'].fillna('').apply(str)
sentences = df2['text'].progress_apply(lambda x: x.split()).values
vocab = build_vocab(sentences)
print({k: vocab[k] for k in list(vocab)[:5]})

100%|██████████| 5/5 [00:00<00:00, 28455.25it/s]

{'I': 1, 'Pooh': 1, '-': 2, 'In': 1, 'silenzio': 1}


In [51]:
def preprocessing (tweet):
  #Remove RT
  tweet2 = re.sub(r'^RT[\s]+', '', tweet)
  #  Replace hyperlinks with URL
  tweet2 = re.sub(r'https?:\/\/.*[\r\n]*', ' url ', tweet2)
  #  Replace hashtags with 'hashtag' 
  tweet2 = re.sub(r'#', ' hashtag ', tweet2)
  # Replace @User with user
  tweet2 = re.sub(r'@[A-Za-z0-9]+', ' user ', tweet2)
  #Remove \n from text
  tweet2 = re.sub(r'\n', '', tweet2)
  #Replace date values with date
  tweet2 = re.sub(r'(\d+/\d+/\d+)', 'date', tweet2)
  #Remove the connectors for the emoji's
  tweet2 = re.sub(r'_', ' ', tweet2)
  tweet2 = re.sub(r'-', ' ', tweet2)
  # Replace numeric terms in the tweet with 'number'.
  tweet2 = re.sub(r'/^[+-]?((\d+(\.\d*)?)|(\.\d+))$/', ' number ', tweet2)
  return (tweet2)


def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


mispell_dict = {"i’m":"i am",
                "isn’t":"is not",
                "it’s":"it is",
                "don’t":"do not",
                "can’t":"can not",
                "we're":"we are",
                "that's":"that is",
                "i've":"i have",
                "you’re":"you are",
                "he’s":"he is",
                "couldn't":"could not",
                "wouldn't":"would not",
                "shouldn't":"should not",
                "ain't":"are not",
                }
mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

def clean_numbers(x):
    #replace number from text with #
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    x = re.sub('[0-9]', '#', x)
    return x

def clean_text(x):
   #remove punctuation from text
    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x
df2['text'] = df2['text'].fillna('').apply(str) 
df2['text'] = df2['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))#apply emoji transformer
df2['text'] = df2['text'].progress_apply(lambda x: replace_typical_misspell(x))#apply preprocessing concatenated words into seperate words
df2['text'] = df2['text'].apply(preprocessing)#apply preprocessing
df2['text'] = df2['text'].apply(lambda x: clean_text(x))
df2['text'] = df2['text'].progress_apply(lambda x: clean_numbers(x)) #apply preprocessing to transform numbers
df2['text'] = df2['text'].apply(lambda x:x.lower())#lower text
df2['text'] = df2['text'].progress_apply(lambda x: word_tokenize(x)) #tokenise text

100%|██████████| 5/5 [00:00<00:00, 3685.68it/s]


In [16]:
vocab = build_vocab(df2['text']) 
oov = check_coverage(vocab,glove_vec) # compares all of tweets with words in word embedding 

100%|██████████| 17/17 [00:00<00:00, 37468.82it/s]

Found embeddings for 94.12% of vocab
Found embeddings for  95.83% of all text


In [17]:
oov[:20]#provides list of words that are in tweets and not in word embedding

[('baccalã', 1)]

In [52]:
df2["Text_Sim"] = df2['text'].progress_apply(lambda x: mean_embeddings(x)) # extract 'embeddings' for each group
h = []
df2 = df2.dropna(subset=['Text_Sim']) #drop rows that have NAN values
s = df2.groupby("user_id")
h = s['Text_Sim'].apply(np.stack).apply(cosine_similarity).apply(np.mean).reset_index()
#  .apply(np.stack) # turns sequence of arrays into proper matrix
#  .apply(cosine_similarity) # compute pairwise similarity matrix
#  .apply(np.mean) # get the mean

100%|██████████| 5/5 [00:00<00:00, 2069.83it/s]


In [53]:
df1=df1.groupby("user_id").mean().reset_index() # calculate average of Tweet content information
df2=df2.groupby("user_id").mean().reset_index() # calculate average of semantic analysis of Tweet information

In [54]:
#concat users profile info, processed Tweet Content, and Tweet Semantic Analysis
df1 = pd.merge(h,df1,on=["user_id"])
df1 = pd.merge(df2,df1,on=["user_id"])
df1 = df1.rename(columns={"user_id": "id"})#rename dataframe column 
df1['id'] = df1['id'].astype("float").astype("Int64")
df1 = df1.drop(columns=['is_bot'])
users = users.merge(df1,how='left', left_on="id", right_on="id")#merge so no information is lost

In [ ]:
users # should be concated dataframe of all features

In [ ]:
df = users
df.to_csv('/content/drive/My Drive/preprocess.csv',encoding='utf-8',index=False) #create processed dataframe for machine learning model